In [1]:
import json
import os

In [2]:
dataset_dir = "Annotations"

In [14]:
def labelme_to_yolo(labelme_json_path, output_file_path):
    with open(labelme_json_path, 'r') as f:
        data = json.load(f)
    
    # Extract image width and height from JSON file
    image_width = data['imageWidth']
    image_height = data['imageHeight']
    
    yolo_annotations = []

    for shape in data['shapes']:
        label = shape['label']
        if label == "thorax":
            category_id = 0  # Assign a unique ID for the "thorax" class
        xmin, ymin = shape['points'][0]
        xmax, ymax = shape['points'][1]

        # Convert bounding box coordinates to YOLO format (normalized)
        x_center = (xmin + xmax) / (2 * image_width)
        y_center = (ymin + ymax) / (2 * image_height)
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height

        # Append YOLO annotation to the list
        yolo_annotations.append((category_id, x_center, y_center, width, height))

    # Save annotations to file
    with open(output_file_path, 'w') as f:
        for annotation in yolo_annotations:
            label, x_center, y_center, width, height = annotation
            f.write(f"{label} {x_center} {y_center} {width} {height}\n")

In [15]:
for d in ["train/labelme_labels", "val/labelme_labels"]:
    for idx, json_file in enumerate(os.listdir(os.path.join(dataset_dir, d))):
        labelme_json_path = os.path.join(dataset_dir, d, json_file)
       
        # Split the directory and file name
        directory, filename = os.path.split(labelme_json_path)
        
        # Replace .json extension with .txt
        base_name, _ = os.path.splitext(filename)
        output_file_path = os.path.join(directory.replace("labelme_labels", "labels"), base_name + '.txt')

        labelme_to_yolo(labelme_json_path, output_file_path)